In [1]:
def calBitOps(num_samples_each_image, image_size=(800, 800)):
    """
    Only counting multiplication
    """
    bitset = [32] * 15
    T = 1e12
    sum = 0
    ray_num = image_size[0] * image_size[1]
    per_ray_bitOps = 0  # 每条光线需要的bitops
    per_point_bitOps = 0  # 每个采样点需要的bitops

    # (1)------Interpolation------
    # ------Trilinear Interpolation------
    L = 24
    F = 4
    hash_bit = bitset[1]
    cord_bit = 32
    TriInterOps = 0
    TriInterOps += 3 * cord_bit * cord_bit
    TriInterOps += L * F * ((4 + 8) * cord_bit * cord_bit + 8 * cord_bit * hash_bit)
    per_point_bitOps += TriInterOps
    # print('TriInterOps per point:',TriInterOps)

    # # (1-2)------Bilinear Interpolation------
    # L = 4
    # F = 2
    # hash_bit = 1
    # cord_bit = 32
    # BiInterOps = 0
    # BiInterOps += 2 * cord_bit * cord_bit
    # BiInterOps += L * F * ((2 + 4) * cord_bit * cord_bit + 4 * cord_bit * hash_bit)
    # per_point_bitOps += BiInterOps * 3
    # # print('BiInterOps per point:',BiInterOps)

    # (2)------SigmaNet------
    encoder_qo_bit = bitset[0]
    sigmanet0_qw_bit = bitset[3]
    sigmanet1_q0_bit = bitset[4]
    sigmanet2_qw_bit = bitset[6]
    sigma_in_dim = 96
    sigma0_dim = 128
    sigma2_dim = 16
    sigma_net_BitOps = 0
    # layer 0
    sigma_net_BitOps += (sigma_in_dim * sigma0_dim) * (encoder_qo_bit * sigmanet0_qw_bit)
    # layer 2
    sigma_net_BitOps += (sigma0_dim * sigma2_dim) * (sigmanet1_q0_bit * sigmanet2_qw_bit)
    per_point_bitOps += sigma_net_BitOps
    # print('sigma_net_BitOps per point:',sigma_net_BitOps)

    # (3)------ColorNet------
    # ColorNet：
    sigma_out_bit = bitset[5]
    dir_bit = bitset[7]
    color0_qw_bit = bitset[10]
    color1_qo_bit = bitset[11]
    color2_qw_bit = bitset[12]
    color3_qo_bit = bitset[13]
    color4_qw_bit = bitset[14]

    sigma_out_dim = 16
    dir_dim = 27
    color0_dim = 128
    color2_dim = 128
    color4_dim = 3

    color_net_BitOps = 0
    # layer 0
    color_net_BitOps += (sigma_out_dim * color0_dim) * (sigma_out_bit * color0_qw_bit)
    color_net_BitOps += dir_dim * color0_dim * dir_bit * color0_qw_bit
    # layer 2
    color_net_BitOps += (color0_dim * color2_dim) * (color1_qo_bit * color2_qw_bit)
    # layer 4
    color_net_BitOps += (color2_dim * color4_dim) * (color3_qo_bit * color4_qw_bit)
    per_point_bitOps += color_net_BitOps
    # print('color_net_BitOps per point:',color_net_BitOps)

    # (4)------volume rendering------
    sigma_bit = bitset[2]
    color_bit = bitset[9]
    volRenderOps = 0
    num_samples_per_ray = num_samples_each_image / ray_num
    # print('samples per ray:',num_samples_per_ray)
    volRenderOps += sigma_bit * num_samples_per_ray * (num_samples_per_ray - 1) / 2
    volRenderOps += num_samples_per_ray * sigma_bit * color_bit
    # print('volRenderOps per ray:',volRenderOps)

    # ------Sum------
    per_ray_bitOps = per_point_bitOps * num_samples_per_ray + volRenderOps
    sum = per_ray_bitOps * ray_num
    # print('bitOps per image:',sum / T, 'T')
    return sum / T


In [3]:
num_samples_each_image = [13214609.64 ,10972904.52 ,4727605.88 ,34744566.92 ,18864164.96 ,16628245.88 ,5466963.12 ,55639113.76 ]
num_samples_each_image = [51901992.32,22536553.44,18781242.40,120730347.80,74008646.56,33240480.48,21966217.44,102872072.20 ]
for i in num_samples_each_image:
    print(calBitOps(i))

2047.9513846963146
889.2318974228508
741.0556420053368
4763.991787440343
2920.277745167715
1311.5890178987647
866.7276567758704
4059.262407102462


# REFERENCE

clc
clear all

% bitset = [8,8,32,8,8,8,8,8,8,8,8,8,8,8,8]; % FQR 9.6 (MDL)
% bitset = [6,6,30,6,6,6,6,6,6,6,6,6,6,6,6]; % FQR 7.6 
% bitset = [5,5,29,5,5,5,5,5,5,5,5,5,5,5,5]; % FQR 6.6 (MGL 32dB)
% bitset = [4,4,28,4,4,4,4,4,4,4,4,4,4,4,4]; % FQR 5.6 (MGL 30dB)
% bitset = repmat(32,1,15);

% ==== A-QAT MDL ====
% bitset = [7, 7, 16, 6, 6, 9, 7, 5, 9, 6, 7, 8, 8, 6, 7]; % chair;
% bitset = [8, 7, 16, 7, 7, 10, 7, 6, 9, 5, 8, 8, 8, 6, 8]; % drums
% bitset = [8, 7, 15, 7, 7, 9, 7, 5, 9, 5, 7, 8, 8, 6, 7]; % ficus
% bitset = [7, 7, 22, 7, 6, 9, 8, 4, 10, 6, 8, 7, 8, 7, 7]; % hotdog
% bitset = [8, 8, 17, 7, 7, 9, 6, 5, 10, 7, 8, 8, 8, 7, 8]; % lego
% bitset = [8, 7, 13, 6, 7, 9, 7, 5, 8, 5, 8, 8, 7, 6, 7]; % materials
% bitset = [9, 8, 16, 7, 8, 10, 8, 5, 10, 6, 8, 8, 8, 6, 6]; % mic
% bitset = [7, 6, 15, 6, 6, 8, 6, 5, 8, 6, 6, 8, 7, 6, 8]; % ship

% ==== A-QAT MGL 10^(-3.2) ====
% bitset = [4, 4, 15, 3, 3, 5, 4, 3, 6, 5, 5, 4, 5, 3, 3]; % chair;
% bitset = [7, 7, 16, 6, 6, 9, 6, 6, 9, 5, 8, 8, 8, 5, 7]; % drums
% bitset = [4, 4, 16, 4, 3, 5, 5, 3, 5, 5, 5, 4, 5, 3, 3]; % ficus
% bitset = [3, 3, 21, 3, 3, 5, 4, 3, 6, 4, 3, 4, 4, 3, 4]; % hotdog
% bitset = [4, 4, 17, 4, 3, 5, 4, 3, 6, 5, 6, 4, 6, 4, 5]; % lego
% bitset = [5, 4, 13, 5, 4, 6, 5, 3, 6, 5, 7, 5, 5, 3, 5]; % materials
% bitset = [3, 3, 16, 3, 3, 5, 3, 3, 5, 5, 4, 4, 5, 3, 3]; % mic
% bitset = [4, 4, 15, 4, 3, 6, 4, 3, 6, 5, 5, 5, 6, 3, 5]; % ship

% ==== A-QAT MGL 10^(-3) ====
% bitset = [4, 3, 18, 4, 3, 5, 4, 3, 5, 5, 4, 3, 4, 3, 3]; % chair;
% bitset = [5, 5, 15, 5, 5, 7, 5, 4, 6, 4, 7, 6, 6, 4, 6]; % drums
% bitset = [3, 3, 17, 3, 3, 5, 3, 3, 4, 5, 3, 3, 3, 3, 3]; % ficus
% bitset = [3, 3, 22, 3, 3, 5, 4, 3, 6, 5, 3, 3, 4, 3, 3]; % hotdog
% bitset = [3, 3, 15, 3, 3, 4, 3, 3, 6, 5, 5, 4, 5, 3, 4]; % lego
% bitset = [4, 4, 13, 4, 3, 6, 4, 3, 5, 6, 5, 4, 6, 3, 5]; % materials
% bitset = [3, 3, 14, 3, 3, 4, 3, 3, 6, 5, 3, 3, 4, 3, 3]; % mic
bitset = [3, 3, 15, 3, 3, 5, 4, 3, 6, 4, 3, 3, 5, 3, 3]; % ship

BitOps = 0;  % final results
model_size = 0; % bits

MB = 1024 * 1024 * 8;
GBITOPS = 1e9;
TBITOPS = 1e12;

% samples_perray = 1024;
image_H = 800;
image_W = 800;

%% configuration for each dataset
% % synthetic: chair
% n_dead = [519971, 10413, 27795, 27899, 17969, 8503, 5009, 3884, 3197, 3091, 3108, 2546, 1949, 1518, 1040, 719, 575, 410, 218, 108, 45, 31, 2];
% n_step = [1, 5, 5, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8];
% hash_size = 6098120 * 2;

% % synthetic: drums
% n_dead = [488529, 11290, 27531, 35153, 32534, 17762, 10579, 6450, 4385, 2546, 1413, 888, 534, 256, 76, 45, 23, 5, 1];
% n_step = [1, 4, 4, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8];
% hash_size = 6098120 * 2;

%synthetic: ficus
% n_dead = [524611, 13012, 23111, 27260, 17836, 11900, 8037, 4823, 2766, 1724, 1432, 1023, 904, 611, 430, 371, 104, 43, 2];
% n_step = [1, 5, 6, repmat(8, 1, 16)];
% hash_size = 6098120 * 2;

% %synthetic: hotdog
% n_dead = [448419, 3607, 18715, 31830, 39261, 31089, 24742, 18231, 10087, 5658, 3928, 2041, 1388, 701, 172, 89, 38, 4];
% n_step = [1, 3, 3, 3, 4, 6, repmat(8, 1, 12)];
% hash_size = 6157376 * 2;

% synthetic: lego
% n_dead = [428694, 7948, 40537, 43309, 43509, 36240, 17193, 8750, 5530, 3470, 1926, 1135, 758, 418, 254, 146, 83, 51, 19, 11, 8, 5, 2, 4];
% n_step = [1, 3, 3, 3, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8];
% hash_size = 6098120 * 2;

% %synthetic: materials
% n_dead = [405374, 5182, 4317, 4202, 4419, 4567, 6954, 8861, 11814, 12344, 11970, 16938, 16056, 18258, 17482, 22656, 24035, 20178, 14176, 7018, 2508, 612, 69];
% n_step = [1, repmat(2, 1, 5), repmat(3, 1, 5), 4, 4, 5, 5, 7, repmat(8, 1, 7)];
% hash_size = 6098120 * 2;

% %synthetic: mic
% n_dead = [538146, 10316, 19886, 17942, 9797, 10220, 9487, 6563, 6272, 5908, 3703, 1421, 266, 38, 19, 8, 5, 2, 1];
% n_step = [1, 6, 6, repmat(8, 1, 16)];
% hash_size = 6098120 * 2;

% % synthetic: ship
n_dead = [311365, 957, 907, 1406, 2219, 3886, 10206, 11081, 11407, 9537, ...
          9704, 11311, 9224, 8727, 7266, 5620, 4799, 4310, 4008, 5636, ...
          5431, 5179, 5430, 5410, 5876, 5858, 6340, 6755, 7108, 9665, ...
          10433, 11053, 15112, 15400, 20427, 20691, 17171, 12962, 8999, ...
          5298, 2811, 1520, 810, 127, 75, 44, 21, 8, 2];
n_step = [ones(1, 5), repmat(2, 1, 13), repmat(3, 1, 10), ...
          repmat(4, 1, 3), repmat(5, 1, 2), 7, repmat(8, 1, 15)];
hash_size = 6157376 * 2;

%% calculate average samples per ray
n_samples = cumsum(n_step);
samples_perray = sum(n_dead.*n_samples)/(image_H * image_W);

%% for one queried pixel:
% multiplication only
% addition & activation are ignroed
%% 1) Trilinear Interpolation
BitOps_temp = 0;
L = 16; % levels
F = 2; % features
cord_bit = 32; % coordiante 
hash_bit = bitset(2);

BitOps_temp = BitOps_temp + 3 * cord_bit * cord_bit;
BitOps_temp = BitOps_temp + L * F * ( (4+8)*cord_bit*cord_bit ...
                                  +      8 *cord_bit*hash_bit );
BitOps = BitOps + BitOps_temp;

model_size = model_size + hash_bit * hash_size;

%% 2) SigmaNet 
BitOps_temp = 0;
encoder_qo_bit = bitset(1);
sigmanet0_qw_bit = bitset(4);
sigmanet1_q0_bit = bitset(5);
sigmanet2_qw_bit = bitset(7);

sigma_in_dim = 32;
sigma0_dim = 64;
sigma2_dim = 16;
% ===== layer 0 =====
BitOps_temp = BitOps_temp + ...
              sigma_in_dim * sigma0_dim * encoder_qo_bit * sigmanet0_qw_bit;
% ===== layer 2 =====
BitOps_temp = BitOps_temp + ...
              sigma0_dim * sigma2_dim * sigmanet1_q0_bit * sigmanet2_qw_bit;

BitOps = BitOps + BitOps_temp;
model_size = model_size + 32 * 64 * sigmanet0_qw_bit + ...
             64 * 16 * sigmanet2_qw_bit;
%% 3) ColorNet
BitOps_temp = 0;
sigma_out_bit = bitset(6);
dir_bit = bitset(8);
color0_qw_bit = bitset(11);
color1_qo_bit = bitset(12);
color2_qw_bit = bitset(13);
color3_qo_bit = bitset(14);
color4_qw_bit = bitset(15);

sigma_out_dim = 15;
dir_dim = 16;
color0_dim = 64;
color2_dim = 64;
color4_dim = 3;
% ===== layer 0 =====
BitOps_temp = BitOps_temp +...
              sigma_out_dim * color0_dim * sigma_out_bit * color0_qw_bit +...
              dir_dim * color0_dim * dir_bit * color0_qw_bit;

% ===== layer 2 =====
BitOps_temp = BitOps_temp +...
              color0_dim * color2_dim * color1_qo_bit * color2_qw_bit;

% ===== layer 4 =====
BitOps_temp = BitOps_temp +...
              color2_dim * color4_dim * color3_qo_bit * color4_qw_bit;

BitOps = BitOps + BitOps_temp;
model_size = model_size + 31 * 64 * color0_qw_bit + ...
             64 * 64 * color2_qw_bit + 64 * 3 * color4_qw_bit;
%% 4) volume rendering
% query all N samples
BitOps = samples_perray * BitOps;

sigma_bit = bitset(3);
color_bit = bitset(10);

BitOps = BitOps + ...
         sigma_bit ^ 2 * samples_perray * (samples_perray-1) / 2 + ...
         samples_perray * sigma_bit * color_bit;

%% sum for all pixels
BitOps = image_W * image_H * BitOps;
BitOps / TBITOPS
model_size / MB